In [1]:
import keyring

keyring.set_password('dart_api_key', 'JuhongPark', '')

In [3]:
import keyring
import requests as rq
from io import BytesIO
import zipfile

api_key = keyring.get_password('dart_api_key', 'JuhongPark')
codezip_url = f'''https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'''
codezip_data = rq.get(codezip_url)
codezip_data.headers

{'Cache-Control': 'no-cache, no-store', 'Connection': 'keep-alive', 'Set-Cookie': 'WMONID=g2MjvaLPgoa; Expires=Sun, 04-May-2025 16:55:28 GMT; Path=/', 'Pragma': 'no-cache', 'Expires': '0', 'Content-Transfer-Encoding': 'binary', 'Content-Disposition': ': attachment; filename=CORPCODE.zip', 'Date': 'Sat, 04 May 2024 07:55:28 GMT', 'Content-Type': 'application/x-msdownload;charset=UTF-8', 'Content-Length': '1804788'}

In [5]:
codezip_file = zipfile.ZipFile(BytesIO(codezip_data.content))
codezip_file.namelist()

['CORPCODE.xml']

In [10]:
import xmltodict
import json
import pandas as pd

code_data = codezip_file.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(code_data)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result').get('list')
corp_list = pd.DataFrame(data)
corp_list = corp_list[~corp_list.stock_code.isin([None])].reset_index(drop=True)

In [15]:
import pymysql
from sqlalchemy import create_engine

corp_list = corp_list[~corp_list.stock_code.isin(
    [None])].reset_index(drop=True)

engine = create_engine('mysql+pymysql://root:@127.0.0.1:3306/stock_db')
corp_list.to_sql(name='dart_code', con=engine, index=True, if_exists='replace')

3724